In [12]:
import os

In [9]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
save_folder = os.path.join(project_root, 'local_only')
os.makedirs(save_folder, exist_ok=True)

In [10]:
dataset_url = 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_test_fullwiki_v1.json'
file_name = dataset_url.split('/')[-1]
output_path = os.path.join(save_folder, file_name)

In [11]:
%%time

import subprocess
subprocess.run(["curl", "-L", "-o", output_path, dataset_url], check=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 82 44.0M   82 36.1M    0     0  5909k      0  0:00:07  0:00:06  0:00:01 7597k

CPU times: user 7.36 ms, sys: 0 ns, total: 7.36 ms
Wall time: 7 s


100 44.0M  100 44.0M    0     0  6479k      0  0:00:06  0:00:06 --:--:-- 9630k


CompletedProcess(args=['curl', '-L', '-o', '/mnt/f/chatbot_ui_v2/local_only/hotpot_test_fullwiki_v1.json', 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_test_fullwiki_v1.json'], returncode=0)

In [22]:
%%time

dataset_url = 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json'
file_name = dataset_url.split('/')[-1]
output_path = os.path.join(save_folder, file_name)

import subprocess
subprocess.run(["curl", "-L", "-o", output_path, dataset_url], check=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 88 45.2M   88 40.2M    0     0  6623k      0  0:00:06  0:00:06 --:--:-- 8512k

CPU times: user 6.68 ms, sys: 990 μs, total: 7.67 ms
Wall time: 6.96 s


100 45.2M  100 45.2M    0     0  6689k      0  0:00:06  0:00:06 --:--:-- 9204k


CompletedProcess(args=['curl', '-L', '-o', '/mnt/f/chatbot_ui_v2/local_only/hotpot_dev_fullwiki_v1.json', 'http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_dev_fullwiki_v1.json'], returncode=0)

In [17]:
import json

In [24]:
# Open and load JSON content
file =  '/mnt/f/chatbot_ui_v2/local_only/hotpot_test_fullwiki_v1.json'
with open(file, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [25]:
type(data), len(data)

(list, 7405)

In [26]:
data[0]

{'_id': '5adf9ba1554299025d62a2db',
 'question': "What position on the Billboard Top 100 did Alison Moyet's late summer hit achieve?",
 'context': [['The Other Side of Love',
   ['"The Other Side of Love" is a song by the British synthpop band Yazoo, released in 1982 as their third single.',
    " The single peaked at #13 on the UK Singles Chart, making it the band's least successful single and the only one of their four singles to miss the top 10.",
    ' The track was written by band members Vince Clarke and Alison Moyet, and was originally not included on either of the band\'s albums (it was later added to a reissue of "Upstairs at Eric\'s").',
    " It featured Stiff Records' all-girl band Sylvia and the Sapphires on backing vocals following a chance meeting on the B.A. Robertson show."]],
  ['All Cried Out (Alison Moyet song)',
   ['"All Cried Out" is a song by English singer-songwriter Alison Moyet.',
    ' It was written by Moyet and producers Jolley & Swain for her debut studio

In [27]:
# Open and load JSON content
file =  '/mnt/f/chatbot_ui_v2/local_only/hotpot_dev_fullwiki_v1.json'
with open(file, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [28]:
type(data), len(data)

(list, 7405)

In [55]:
def process_record(record):
    data = dict(record)
    context = data.pop('context')
    condensed_context = condense_context(context)
    _id = data.pop('_id')
    supporting_facts = data.pop('supporting_facts')
    supporting_facts = list({i[0] for i in supporting_facts})
    metadata = dict(answer = data.pop('answer'), type = data.pop('type'), level=data.pop('level'), 
                    question = data.pop('question'), supporting_facts=supporting_facts)

    documents = list(condensed_context.values())
    ids = [f'{_id}_{i}' for i in range(len(documents))]
    metadatas = [dict(**metadata, title=title)   for title in condensed_context.keys()]
    
    return dict(documents=documents, metadatas=metadatas, ids=ids)

In [58]:
process_record(data[1])

{'documents': ['A Kiss for Corliss is a 1949 American comedy film directed by Richard Wallace and written by Howard Dimsdale. It stars Shirley Temple in her final starring role as well as her final film appearance. It is a sequel to the 1945 film "Kiss and Tell". "A Kiss for Corliss" was retitled "Almost a Bride" before release and this title appears in the title sequence. The film was released on November 25, 1949, by United Artists.',
  'The post of Lord High Treasurer or Lord Treasurer was an English government position and has been a British government position since the Acts of Union of 1707. A holder of the post would be the third-highest-ranked Great Officer of State, below the Lord High Steward and the Lord High Chancellor.',
  'Meet Corliss Archer is an American television sitcom that aired on CBS (July 13, 1951 - August 10, 1951) and in syndication via the Ziv Company from April to December 1954. The program was an adaptation of the radio series of the same name, which was ba

In [43]:
def condense_context(context):
    return {i[0]: ''.join(i[1]) for i in context}

In [59]:
import chromadb

In [60]:
client = chromadb.HttpClient(
    host="localhost",
    port=int(8010))

In [61]:
dir(client)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add',
 '_admin_client',
 '_count',
 '_create_system_if_not_exists',
 '_delete',
 '_get',
 '_get_identifier_from_settings',
 '_identifier',
 '_identifier_to_system',
 '_modify',
 '_peek',
 '_populate_data_from_system',
 '_query',
 '_server',
 '_submit_client_start_event',
 '_system',
 '_update',
 '_upsert',
 '_validate_tenant_database',
 'clear_system_cache',
 'count_collections',
 'create_collection',
 'database',
 'delete_collection',
 'from_system',
 'get_collection',
 'get_max_batch_size',
 'get_or_create_collection',
 'get_settings',
 'get_user_

In [62]:
client.list_collections()

[Collection(name=test_collection)]

In [68]:
collection = client.get_collection('test_collection')

In [70]:
collection.id

UUID('935215a8-24d7-466b-ac31-400c97df58cb')

In [71]:
collection.get()

{'ids': ['5a8b57f25542995d1e6f1371_0',
  '5a8b57f25542995d1e6f1371_1',
  '5a8b57f25542995d1e6f1371_2',
  '5a8b57f25542995d1e6f1371_3',
  '5a8b57f25542995d1e6f1371_4',
  '5a8b57f25542995d1e6f1371_5',
  '5a8b57f25542995d1e6f1371_6',
  '5a8b57f25542995d1e6f1371_7',
  '5a8b57f25542995d1e6f1371_8',
  '5a8b57f25542995d1e6f1371_9',
  '5a8c7595554299585d9e36b6_0',
  '5a8c7595554299585d9e36b6_1',
  '5a8c7595554299585d9e36b6_2',
  '5a8c7595554299585d9e36b6_3',
  '5a8c7595554299585d9e36b6_4',
  '5a8c7595554299585d9e36b6_5',
  '5a8c7595554299585d9e36b6_6',
  '5a8c7595554299585d9e36b6_7',
  '5a8c7595554299585d9e36b6_8',
  '5a8c7595554299585d9e36b6_9',
  '5a85ea095542994775f606a8_0',
  '5a85ea095542994775f606a8_1',
  '5a85ea095542994775f606a8_2',
  '5a85ea095542994775f606a8_3',
  '5a85ea095542994775f606a8_4',
  '5a85ea095542994775f606a8_5',
  '5a85ea095542994775f606a8_6',
  '5a85ea095542994775f606a8_7',
  '5a85ea095542994775f606a8_8',
  '5a85ea095542994775f606a8_9',
  '5adbf0a255429947ff17385a_0',
 

In [76]:
abc = collection.get(include=["embeddings"])

In [78]:
abc

{'ids': ['5a8b57f25542995d1e6f1371_0',
  '5a8b57f25542995d1e6f1371_1',
  '5a8b57f25542995d1e6f1371_2',
  '5a8b57f25542995d1e6f1371_3',
  '5a8b57f25542995d1e6f1371_4',
  '5a8b57f25542995d1e6f1371_5',
  '5a8b57f25542995d1e6f1371_6',
  '5a8b57f25542995d1e6f1371_7',
  '5a8b57f25542995d1e6f1371_8',
  '5a8b57f25542995d1e6f1371_9',
  '5a8c7595554299585d9e36b6_0',
  '5a8c7595554299585d9e36b6_1',
  '5a8c7595554299585d9e36b6_2',
  '5a8c7595554299585d9e36b6_3',
  '5a8c7595554299585d9e36b6_4',
  '5a8c7595554299585d9e36b6_5',
  '5a8c7595554299585d9e36b6_6',
  '5a8c7595554299585d9e36b6_7',
  '5a8c7595554299585d9e36b6_8',
  '5a8c7595554299585d9e36b6_9',
  '5a85ea095542994775f606a8_0',
  '5a85ea095542994775f606a8_1',
  '5a85ea095542994775f606a8_2',
  '5a85ea095542994775f606a8_3',
  '5a85ea095542994775f606a8_4',
  '5a85ea095542994775f606a8_5',
  '5a85ea095542994775f606a8_6',
  '5a85ea095542994775f606a8_7',
  '5a85ea095542994775f606a8_8',
  '5a85ea095542994775f606a8_9',
  '5adbf0a255429947ff17385a_0',
 